In [ ]:
# Uploadowanie do przestrzeni roboczej 

from google.colab import files

uploaded = files.upload()


In [ ]:
# Wczytanie bazy obrazow
import os
from zipfile import ZipFile
import cv2
import numpy as np

def readCPOdataset(datasetPath):
  with ZipFile(datasetPath, 'r') as zip:
    zip.extractall()
  folder = os.path.dirname(datasetPath)  

  trainImg = [os.path.join(folder + '/train_img', f) for f in os.listdir(folder + '/train_img') if os.path.isfile(os.path.join(folder + '/train_img', f))]
  testImg = [os.path.join(folder + '/test_img', f) for f in os.listdir(folder + '/test_img') if os.path.isfile(os.path.join(folder + '/test_img', f))]
  trainImg.sort()
  testImg.sort() 

  def readLabelFile(path):
    labels = []
    with open(path,'r') as f:
      labels = f.readlines()
      labels = [int(x.rstrip('\n')) for x in labels]
    return labels 
  trainLabels = readLabelFile(folder + '/train.txt')
  testLabels = readLabelFile(folder + '/test.txt')  

  return trainImg, testImg, trainLabels, testLabels

trainImg, testImg, trainLabels, testLabels = readCPOdataset('/content/dataset.zip')
print(f"Sciezki do danych uczacych: {trainImg}\Sciezki do danych walidacyjnych: {testImg}")
print(f"Etykiety danych uczacych: {trainLabels}\Etykiety danych walidacyjnych: {testLabels}")

Sciezki do danych uczacych: ['/content/train_img/00.png', '/content/train_img/01.png', '/content/train_img/02.png', '/content/train_img/03.png', '/content/train_img/04.png', '/content/train_img/05.png', '/content/train_img/06.png', '/content/train_img/07.png', '/content/train_img/08.png', '/content/train_img/09.png', '/content/train_img/10.png', '/content/train_img/11.png', '/content/train_img/12.png', '/content/train_img/13.png', '/content/train_img/14.png', '/content/train_img/15.png', '/content/train_img/16.png', '/content/train_img/17.png', '/content/train_img/18.png', '/content/train_img/19.png', '/content/train_img/20.png', '/content/train_img/21.png', '/content/train_img/22.png', '/content/train_img/23.png', '/content/train_img/24.png', '/content/train_img/25.png', '/content/train_img/26.png', '/content/train_img/27.png', '/content/train_img/28.png', '/content/train_img/29.png', '/content/train_img/30.png', '/content/train_img/31.png', '/content/train_img/32.png', '/content/trai

In [ ]:
from skimage.feature import hog
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn import svm


# Wstępne przetwarzanie obrazów
def preprocess(images, size):
    # Zmniejszenie rozmiaru obrazów
    resized_images = []
    for img in images:
        resized_images.append(cv2.resize(cv2.imread(img), size))
    # Konwersja obrazów do skali szarości
    gray_images = []
    for img in resized_images:
        gray_images.append(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY))
    return gray_images

# Wyznaczenie wektora cech metodą HOG
def extract_hog_features(images, orientations, pixels_per_cell, cells_per_block):
    features = []
    for img in images:
        hog_features = hog(img, orientations=orientations, pixels_per_cell=pixels_per_cell, cells_per_block=cells_per_block)
        features.append(hog_features)
    return features

# Wczytanie i wstępne przetworzenie obrazów
trainImages = preprocess(trainImg, (128, 64))
testImages = preprocess(testImg, (128, 64))

# Wyznaczenie wektora cech metodą HOG dla całego obrazu
orientations = 10
pixels_per_cell = (16, 16)
cells_per_block = (2, 2)

trainFeatures = extract_hog_features(trainImages, orientations, pixels_per_cell, cells_per_block)
testFeatures = extract_hog_features(testImages, orientations, pixels_per_cell, cells_per_block)

# Klasyfikacja metodą SVM na podstawie wyznaczonych wektorów cech i informacji o klasach obiektów
clf = LinearSVC()
clf.fit(trainFeatures, trainLabels )
predictedLabels = clf.predict(testFeatures)

# Ocena wyników

# W klasyfikacji wieloetykietowej accuracy_score oblicza dokładność podzbioru: 
# zestaw etykiet przewidziany dla próbki musi dokładnie pasować do odpowiedniego 
# zestawu etykiet w trainLabels / testLabels

trainAccuracy = accuracy_score(trainLabels, clf.predict(trainFeatures))
testAccuracy = accuracy_score(testLabels, predictedLabels)


print("Dokładność dla danych treningowych: {:.2f}%".format(trainAccuracy * 100))
print("Dokładność dla danych testowych: {:.2f}%".format(testAccuracy * 100))


Dokładność dla danych treningowych: 100.00%
Dokładność dla danych testowych: 85.00%


In [ ]:

# preprocesing danych uczących i walidacyjnych
trainData = []
for imgPath in trainImg:
  img = cv2.imread(imgPath, cv2.IMREAD_GRAYSCALE)
  # binarization
  img = cv2.threshold(img, 128, 255, cv2.THRESH_BINARY)[1]
  # deskew
  moments = cv2.moments(img)
  skew = moments['mu11'] / moments['mu02']
  M = np.float32([[1, skew, -0.5 * 28 * skew], [0, 1, 0]])
  img = cv2.warpAffine(img, M, (28, 28), flags=cv2.WARP_INVERSE_MAP | cv2.INTER_LINEAR)
  # resize
  img = cv2.resize(img, (20, 20), interpolation=cv2.INTER_AREA)
  trainData.append(img.flatten())

testData = []
for imgPath in testImg:
  img = cv2.imread(imgPath, cv2.IMREAD_GRAYSCALE)
  # binarization
  img = cv2.threshold(img, 128, 255, cv2.THRESH_BINARY)[1]
  # deskew
  moments = cv2.moments(img)
  skew = moments['mu11'] / moments['mu02']
  M = np.float32([[1, skew, -0.5 * 28 * skew], [0, 1, 0]])
  img = cv2.warpAffine(img, M, (28, 28), flags=cv2.WARP_INVERSE_MAP | cv2.INTER_LINEAR)
  # resize
  img = cv2.resize(img, (20, 20), interpolation=cv2.INTER_AREA)
  testData.append(img.flatten())

# konwersja danych uczących i walidacyjnych na numpy array
trainData = np.array(trainData)
testData = np.array(testData)

# klasyfikacja kNN
k = 3
distances = np.zeros((testData.shape[0], trainData.shape[0]))
for i, testImage in enumerate(testData):
  for j, trainImage in enumerate(trainData):
    distances[i,j] = np.sum(np.abs(testImage - trainImage))
nearestNeighbors = np.argsort(distances, axis=1)[:, :k]
nearestLabels = np.take(trainLabels, nearestNeighbors)
predictedLabels = np.zeros(testData.shape[0], dtype=np.int)
for i in range(testData.shape[0]):
  predictedLabels[i] = np.bincount(nearestLabels[i]).argmax()

# obliczenie skuteczności klasyfikacji
accuracy = np.sum(predictedLabels == testLabels) / len(testLabels)
print(f"Skuteczność klasyfikacji: {accuracy:.2f}")

Skuteczność klasyfikacji: 0.60


<ipython-input-19-312fe50bc042>:42: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  predictedLabels = np.zeros(testData.shape[0], dtype=np.int)
